In [ ]:
import sys, os
nb_dir = os.getcwd()
# 2) Eine Ebene hoch → …/fairfluence/src
src_dir = os.path.abspath(os.path.join(nb_dir, os.pardir))
# 3) Noch eine Ebene hoch → …/fairfluence
project_root = os.path.abspath(os.path.join(src_dir, os.pardir))

# 4) Füge fairfluence/ nach ganz vorne in sys.path
if project_root not in sys.path:
    sys.path.insert(0, project_root)
    
import pandas as pd
from preprocessing import PreprocessorFactory, Preprocessor
import pytest
from datasets_dict import DATASETS
from src.ingestion.ingestorFactory import IngestorFactory

c:\Uni\Master\Semester_2\EDML\Project\fairfluence\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
import sys, os
nb_dir = os.getcwd()
# 2) Eine Ebene hoch → …/fairfluence/src
src_dir = os.path.abspath(os.path.join(nb_dir, os.pardir))
# 3) Noch eine Ebene hoch → …/fairfluence
project_root = os.path.abspath(os.path.join(src_dir, os.pardir))

# 4) Füge fairfluence/ nach ganz vorne in sys.path
if project_root not in sys.path:
    sys.path.insert(0, project_root)
    
import pandas as pd
from preprocessing import PreprocessorFactory, Preprocessor
import pytest
from datasets_dict import DATASETS
from src.ingestion.ingestorFactory import IngestorFactory

def run_preprocessor_checks():
    methods = ["data quality", "fairness"]
    ohe_flags = [False, True]

    for index, data in enumerate(DATASETS.items()):
        data_link = data[0]
        info = data[1]
        if data_link in ["https://www.kaggle.com/datasets/aemreusta/brfss-2020-survey-data",
                        "https://www.kaggle.com/datasets/fleshmetal/records-a-comprehensive-music-metadata-dataset",
                        "https://www.kaggle.com/datasets/sudipde25/lifespan-calorie-burn-dynamics"]:
            continue
        target_column = info["target_column"]
        nr_file = info["no_dataset"]
        # 1) CSV laden
        ingestor_factory = IngestorFactory(data_link, nr_file, False)
        ingestor = ingestor_factory.create()
        df = ingestor.load_data()
        print(f"\n==> Testing {data_link!r}  target={target_column}")

        for method in methods:
            for ohe in ohe_flags:
                label = f"  [{method:12s} | ohe={ohe}]"
                try:
                    # 2) Factory + Preprocessor
                    factory = PreprocessorFactory(df, method, target_column)
                    pre     = factory.create()

                    # 3) Aufruf process_data
                    result = pre.process_data(ohe=ohe)

                    # 4) Struktur prüfen
                    if not isinstance(result, tuple):
                        raise AssertionError(f"{label} ➞ returned no tuple")
                    if len(result) != 6:
                        raise AssertionError(f"{label} ➞ expected 6 items, got {len(result)}")

                    df_tr, num_cols, cat_cols, text_feats, sens_cols, tgt_col = result

                    # 5a) DataFrame-Typ und Zeilenzahl
                    if not isinstance(df_tr, pd.DataFrame):
                        raise AssertionError(f"{label} ➞ first item not a DataFrame")
                    if df_tr.shape[0] != df.shape[0]:
                        raise AssertionError(f"{label} ➞ row count changed {df_tr.shape[0]} vs {df.shape[0]}")

                    # 5b) Target-Spalte vorhanden
                    if target_column not in df_tr.columns:
                        raise AssertionError(f"{label} ➞ target column '{target_column}' missing")
                    
                    # Nan Check
                    if df_tr.isnull().values.any():
                        nan_cols = df_tr.columns[df_tr.isnull().any()].tolist()
                        raise AssertionError(f"{label} ➞ found NaNs in columns {nan_cols}")

                    # 6) Die vier Listen müssen Listen sein
                    for name, lst in [
                        ("numeric_columns",    num_cols),
                        ("categorical_columns",cat_cols),
                        ("text_features",      text_feats),
                        ("sensitive_columns",  sens_cols)
                    ]:
                        if not isinstance(lst, list):
                            raise AssertionError(f"{label} ➞ '{name}' is not a list")

                    # 7) target_column-Return
                    if not isinstance(tgt_col, str) or tgt_col != target_column:
                        raise AssertionError(f"{label} ➞ returned target '{tgt_col}'") 
                    
                    vals = df_tr[target_column].unique()
                    if len(vals) != 2:
                        raise AssertionError(
                            f"{label} ➞ target column must be binary, "
                            f"but found values {vals}"
                        )

                    print(f"{label} ✓")
                except Exception as e:
                    print(f"{label} ✗ ERROR: {e}")

    print("\nAll checks done.")


run_preprocessor_checks()

Creating CSV from Arrow format: 100%|██████████| 33/33 [00:00<00:00, 395.40ba/s]



==> Testing 'https://huggingface.co/datasets/scikit-learn/adult-census-income'  target=income
  [data quality | ohe=False] ✓
  [data quality | ohe=True] ✓
  [fairness     | ohe=False] ✓
  [fairness     | ohe=True] ✓
student_habits_performance.csv
Dataset URL: https://www.kaggle.com/datasets/jayaantanaath/student-habits-vs-academic-performance
../../data/kaggle_temp/student_habits_performance.csv was deleted.

==> Testing 'https://www.kaggle.com/datasets/jayaantanaath/student-habits-vs-academic-performance'  target=exam_score
  [data quality | ohe=False] ✓
  [data quality | ohe=True] ✓
  [fairness     | ohe=False] ✓
  [fairness     | ohe=True] ✓
student_info.csv
Dataset URL: https://www.kaggle.com/datasets/therohithanand/student-academic-performance-dataset
../../data/kaggle_temp/student_info.csv was deleted.

==> Testing 'https://www.kaggle.com/datasets/therohithanand/student-academic-performance-dataset'  target=final_result
  [data quality | ohe=False] ✓
  [data quality | ohe=True] 

Creating CSV from Arrow format: 100%|██████████| 100/100 [00:00<00:00, 546.43ba/s]



==> Testing 'https://huggingface.co/datasets/marianeft/diabetes_prediction_dataset'  target=diabetes
  [data quality | ohe=False] ✓
  [data quality | ohe=True] ✓
  [fairness     | ohe=False] ✓
  [fairness     | ohe=True] ✓


Creating CSV from Arrow format: 100%|██████████| 30/30 [00:00<00:00, 198.87ba/s]


...

tested.csv
Dataset URL: https://www.kaggle.com/datasets/brendan45774/test-file
../../data/kaggle_temp/tested.csv was deleted.

==> Testing 'https://www.kaggle.com/datasets/brendan45774/test-file'  target=Survived
  [data quality | ohe=False] ✓
  [data quality | ohe=True] ✓
  [fairness     | ohe=False] ✓
  [fairness     | ohe=True] ✓
speeddating.csv
Dataset URL: https://www.kaggle.com/datasets/ulrikthygepedersen/speed-dating
../../data/kaggle_temp/speeddating.csv was deleted.

==> Testing 'https://www.kaggle.com/datasets/ulrikthygepedersen/speed-dating/data'  target=match
  [data quality | ohe=False] ✓
  [data quality | ohe=True] ✓
  [fairness     | ohe=False] ✓
  [fairness     | ohe=True] ✓
451
<class 'str'>

==> Testing 'https://www.openml.org/search?type=data&sort=runs&status=active&id=451'  target=Leaving_Certificate
  [data quality | ohe=False] ✓
  [data quality | ohe=True] ✓
  [fairness     | ohe=False] ✓
  [fairness     | ohe=True] ✓
42164
<class 'str'>

==> Testing 'https://www.

c:\Uni\Master\Semester_2\EDML\Project\fairfluence\.venv\Lib\site-packages\sklearn\preprocessing\_discretization.py:334: UserWarning: Feature 0 is constant and will be replaced with 0.
  warnings.warn(
c:\Uni\Master\Semester_2\EDML\Project\fairfluence\.venv\Lib\site-packages\sklearn\preprocessing\_discretization.py:334: UserWarning: Feature 0 is constant and will be replaced with 0.
  warnings.warn(


fertility.csv
Dataset URL: https://www.kaggle.com/datasets/muhamedyoussry/fertility-data-set
../../data/kaggle_temp/fertility.csv was deleted.

==> Testing 'https://www.kaggle.com/datasets/muhamedyoussry/fertility-data-set'  target=Diagnosis
  [data quality | ohe=False] ✓
  [data quality | ohe=True] ✓
  [fairness     | ohe=False] ✓
  [fairness     | ohe=True] ✓
physics_teacher_jobs_linkedin.csv
Dataset URL: https://www.kaggle.com/datasets/kushalmanage/physics-teacher-jobs-linkedin
../../data/kaggle_temp/physics_teacher_jobs_linkedin.csv was deleted.

==> Testing 'https://www.kaggle.com/datasets/kushalmanage/physics-teacher-jobs-linkedin'  target=contractType
  [data quality | ohe=False] ✓
  [data quality | ohe=True] ✓
  [fairness     | ohe=False] ✓
  [fairness     | ohe=True] ✓
Public Vs Private Universties.csv
Dataset URL: https://www.kaggle.com/datasets/amosshehzad/public-vs-privat-schools-statistical-analysis
../../data/kaggle_temp/Public Vs Private Universties.csv was deleted.

==>

c:\Uni\Master\Semester_2\EDML\Project\fairfluence\src\ingestion\ingestorFactory.py:380: DtypeWarning: Columns (20,50,161,200) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(final_csv)



==> Testing 'https://www.kaggle.com/datasets/threnjen/portland-housing-prices-sales-jul-2020-jul-2021'  target=price
  [data quality | ohe=False] ✓
  [data quality | ohe=True] ✓
  [fairness     | ohe=False] ✓
  [fairness     | ohe=True] ✓
duplicate_deleted.csv
Dataset URL: https://www.kaggle.com/datasets/janakpariyar/coursera-courses-uncleaned-dataset-to-practice
../../data/kaggle_temp/duplicate_deleted.csv was deleted.

==> Testing 'https://www.kaggle.com/datasets/janakpariyar/coursera-courses-uncleaned-dataset-to-practice'  target=course_rating
  [data quality | ohe=False] ✓
  [data quality | ohe=True] ✓
  [fairness     | ohe=False] ✓
  [fairness     | ohe=True] ✓

All checks done.


In [3]:
import sys, os
nb_dir = os.getcwd()
# 2) Eine Ebene hoch → …/fairfluence/src
src_dir = os.path.abspath(os.path.join(nb_dir, os.pardir))
# 3) Noch eine Ebene hoch → …/fairfluence
project_root = os.path.abspath(os.path.join(src_dir, os.pardir))

# 4) Füge fairfluence/ nach ganz vorne in sys.path
if project_root not in sys.path:
    sys.path.insert(0, project_root)
    
import pandas as pd
from preprocessing import PreprocessorFactory, Preprocessor
import pytest
from datasets_dict import DATASETS
from src.ingestion.ingestorFactory import IngestorFactory

data_link = "https://www.kaggle.com/datasets/ehtishamsadiq/uncleaned-laptop-price-dataset"
nr_file = 1
target_column = "Price"
ingestor_factory = IngestorFactory(data_link, nr_file, False)
ingestor = ingestor_factory.create()
data = ingestor.load_data()
dqp = Preprocessor(data, target_column="")
numeric_columns, categorical_columns = dqp.receive_categorized_columns()
preprocessor_factory = PreprocessorFactory(data, "fairness", target_column=target_column)
preprocessor = preprocessor_factory.create()
processed_data_dict = preprocessor.process_data(ohe=True)


processed_data_dict[0]

laptopData.csv
Dataset URL: https://www.kaggle.com/datasets/ehtishamsadiq/uncleaned-laptop-price-dataset
../../data/kaggle_temp/laptopData.csv was deleted.


,Company_<MISSING>,Company_Acer,Company_Apple,Company_Asus,Company_Chuwi,Company_Dell,Company_Fujitsu,Company_Google,Company_HP,Company_Huawei,...,OpSys_Mac OS X,OpSys_No OS,OpSys_Windows 10,OpSys_Windows 10 S,OpSys_Windows 7,OpSys_macOS,Unnamed: 0,Weight_svd_0,Weight_svd_1,Price
0,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,1,-1.720688,-5.395400e-07,-7.714979e-05,1
1,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,1,-1.720688,2.361558e-10,-3.895210e-07,0
2,0,0,0,0,0,0,0,0,1,0,...,0,1,0,0,0,0,-1.720688,-2.345284e-06,-2.149377e-05,0
3,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,1,-1.720688,2.496907e-11,1.410917e-08,1
4,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,1,-1.720688,-5.395400e-07,-7.714979e-05,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1298,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,1.711762,3.982685e-09,3.332271e-06,0
1299,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,1.711762,-3.471560e-10,1.000000e+00,1
1300,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,1.711762,2.467374e-08,4.572228e-06,0
1301,0,0,0,0,0,0,0,0,1,0,...,0,0,1,0,0,0,1.711762,2.791961e-09,-2.539598e-07,0


In [15]:
target_column in numeric_columns

True

In [16]:
target_column in categorical_columns

False

In [4]:
processed_data_dict[4]

['AGE']

In [4]:
data

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,MEDV
0,0.00632,18.0,2.31,0.0,0.538,6.575,65.2,4.0900,1,296,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0.0,0.469,6.421,78.9,4.9671,2,242,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0.0,0.469,7.185,61.1,4.9671,2,242,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0.0,0.458,6.998,45.8,6.0622,3,222,18.7,394.63,2.94,33.4
4,0.06905,0.0,2.18,0.0,0.458,7.147,54.2,6.0622,3,222,18.7,396.90,NaN,36.2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
501,0.06263,0.0,11.93,0.0,0.573,6.593,69.1,2.4786,1,273,21.0,391.99,NaN,22.4
502,0.04527,0.0,11.93,0.0,0.573,6.120,76.7,2.2875,1,273,21.0,396.90,9.08,20.6
503,0.06076,0.0,11.93,0.0,0.573,6.976,91.0,2.1675,1,273,21.0,396.90,5.64,23.9
504,0.10959,0.0,11.93,0.0,0.573,6.794,89.3,2.3889,1,273,21.0,393.45,6.48,22.0


In [11]:
import sys, os
nb_dir = os.getcwd()
# 2) Eine Ebene hoch → …/fairfluence/src
src_dir = os.path.abspath(os.path.join(nb_dir, os.pardir))
# 3) Noch eine Ebene hoch → …/fairfluence
project_root = os.path.abspath(os.path.join(src_dir, os.pardir))

# 4) Füge fairfluence/ nach ganz vorne in sys.path
if project_root not in sys.path:
    sys.path.insert(0, project_root)
    
import pandas as pd
from preprocessing import PreprocessorFactory
import pytest
from datasets_dict import DATASETS
from src.ingestion.ingestorFactory import IngestorFactory

data_link = "https://huggingface.co/datasets/Supa-AI/Reasoning_Patterns_AI_Hiring_Bias_SEA"
nr_file = 1
target_column = "Is Bias Likely? (Yes/No)"
ingestor_factory = IngestorFactory(data_link, nr_file, False)
ingestor = ingestor_factory.create()
data = ingestor.load_data()
preprocessor_factory = PreprocessorFactory(data, "fairness", target_column=target_column)
preprocessor = preprocessor_factory.create()
processed_data_dict = preprocessor.process_data(ohe=True)
processed_data_dict[0]

Creating CSV from Arrow format: 100%|██████████| 1/1 [00:00<00:00, 326.76ba/s]


,Gender A_Female,Gender A_Male,Gender B_Female,Gender B_Male,Age A_25.00–26.00,Age A_26.00–27.00,Age A_27.00–28.00,Age A_28.00–29.00,Age A_29.00–30.00,Age A_30.00–31.00,...,Experience_svd_1,Chosen Candidate_svd_0,Chosen Candidate_svd_1,Justification_svd_0,Justification_svd_1,Tone Flag_svd_0,Tone Flag_svd_1,Notes_svd_0,Notes_svd_1,Is Bias Likely? (Yes/No)
0,1,0,1,0,0,0,1,0,0,0,...,0.068327,1.000000e+00,-1.150677e-14,0.293339,-0.023429,5.720228e-05,9.994953e-01,0.249894,-0.094869,0
1,1,0,1,0,0,0,1,0,0,0,...,0.068327,2.439675e-13,3.648670e-10,0.220394,0.047070,2.254660e-13,-5.294464e-10,0.192569,-0.090976,3
2,1,0,1,0,0,0,1,0,0,0,...,0.068327,1.000000e+00,-1.150677e-14,0.261938,0.191481,7.920444e-03,2.647180e-03,0.326123,0.200585,0
3,1,0,1,0,0,0,1,0,0,0,...,0.068327,1.150686e-14,1.000000e+00,0.471401,0.459369,9.998407e-01,-9.308093e-05,0.196877,0.009507,0
4,1,0,1,0,0,0,1,0,0,0,...,0.068327,1.000000e+00,-1.150677e-14,0.178301,-0.027427,7.223724e-03,2.724537e-03,0.263340,0.182026,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
178,0,1,0,1,0,0,0,0,0,0,...,-0.175600,1.000000e+00,-1.150677e-14,0.240811,-0.047939,-1.377625e-09,-1.468718e-08,0.110642,-0.129972,0
179,0,1,0,1,0,0,0,0,0,0,...,-0.175600,1.000000e+00,-1.150677e-14,0.164764,-0.013879,5.720228e-05,9.994953e-01,0.142328,-0.145958,1
180,0,1,0,1,0,0,0,0,0,0,...,-0.175600,3.839885e-14,6.044291e-11,0.139952,-0.069865,9.998407e-01,-9.308093e-05,0.243671,-0.251866,3
181,0,1,0,1,0,0,0,0,0,0,...,-0.175600,1.000000e+00,-1.150677e-14,0.258374,-0.114160,2.846645e-03,1.258138e-03,0.344899,0.065878,0


In [12]:
processed_data_dict[3]

['Test Pair ID_svd_0',
 'Test Pair ID_svd_1',
 'Name A_svd_0',
 'Name A_svd_1',
 'Name B_svd_0',
 'Name B_svd_1',
 'Ethnicity A_svd_0',
 'Ethnicity A_svd_1',
 'Ethnicity B_svd_0',
 'Ethnicity B_svd_1',
 'Education B_svd_0',
 'Education B_svd_1',
 'Company A_svd_0',
 'Company A_svd_1',
 'Company B_svd_0',
 'Company B_svd_1',
 'Job Title_svd_0',
 'Job Title_svd_1',
 'Experience_svd_0',
 'Experience_svd_1',
 'Chosen Candidate_svd_0',
 'Chosen Candidate_svd_1',
 'Justification_svd_0',
 'Justification_svd_1',
 'Tone Flag_svd_0',
 'Tone Flag_svd_1',
 'Notes_svd_0',
 'Notes_svd_1']